<a href="https://colab.research.google.com/github/hamzaae/Neural_Network_FeedForword_Diabetes_DataSet/blob/main/Neural_Network_FeedForword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Importing Modules

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

2. Creating DataFrame

In [8]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


3. Visualizing dataset

In [ ]:
for i in range(len(df.columns[:-1])):
  label = df.columns[i]
  plt.hist(df[df['Outcome']==1][label], color='blue', label="Diabetes", alpha=0.7, density=True, bins=15)
  plt.hist(df[df['Outcome']==0][label], color='red', label="No diabetes", alpha=0.7, density=True, bins=15)
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  plt.show()

Scaling and reshaping the quantities for good mesures

In [21]:
X = df[df.columns[:-1]].values
y = df[df.columns[-1]].values
print('Shape before : ',X.shape, y.shape) # X is 2D and y is 1D
print('Length before : ',len(df[df["Outcome"]==1]), len(df[df["Outcome"]==0]))

scaler = StandardScaler()
X = scaler.fit_transform(X)
data = np.hstack((X, np.reshape(y, (-1, 1))))
transformed_df = pd.DataFrame(data, columns=df.columns)

over = RandomOverSampler()
X, y = over.fit_resample(X, y)
data = np.hstack((X, np.reshape(y, (-1, 1))))
transformed_df = pd.DataFrame(data, columns=df.columns)

print('Shape after : ',X.shape, y.shape)
print('Length after : ',len(transformed_df[transformed_df["Outcome"]==1]), len(transformed_df[transformed_df["Outcome"]==0]))

Shape before :  (768, 8) (768,)
Length before :  268 500
Shape after :  (1000, 8) (1000,)
Length after :  500 500


4. Strartring process

Deviding data into:
*   Training 60%
*   Validation 20%
*   Test 20%

In [22]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

Building Model by Sequencing stack of layers using 'Relu' as activation function for Dense layers and 'Sigmoid' for the final Dense layer, and setting 'Adam' optimizer beacause we are doing a binary classification

In [23]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

Evaluating model

In [24]:
model.evaluate(X_train, y_train)

19/19 [==============================] - 0s 2ms/step - loss: 0.7275 - accuracy: 0.4417


[0.7275047898292542, 0.4416666626930237]

In [25]:
model.evaluate(X_valid, y_valid)

7/7 [==============================] - 0s 2ms/step - loss: 0.7535 - accuracy: 0.4700


[0.7534776926040649, 0.4699999988079071]

The accuracy of the model is very bad.
We haven't done any training, let's do it!

In [26]:
model.fit(X_train, y_train, batch_size=16, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
38/38 [==============================] - 1s 4ms/step - loss: 0.6974 - accuracy: 0.5067 - val_loss: 0.6707 - val_accuracy: 0.6100
Epoch 2/20
38/38 [==============================] - 0s 4ms/step - loss: 0.6361 - accuracy: 0.6650 - val_loss: 0.6046 - val_accuracy: 0.7350
Epoch 3/20
38/38 [==============================] - 0s 4ms/step - loss: 0.5865 - accuracy: 0.7217 - val_loss: 0.5526 - val_accuracy: 0.7550
Epoch 4/20
38/38 [==============================] - 0s 5ms/step - loss: 0.5518 - accuracy: 0.7317 - val_loss: 0.5188 - val_accuracy: 0.7800
Epoch 5/20
38/38 [==============================] - 0s 5ms/step - loss: 0.5332 - accuracy: 0.7350 - val_loss: 0.4999 - val_accuracy: 0.7750
Epoch 6/20
38/38 [==============================] - 0s 5ms/step - loss: 0.5191 - accuracy: 0.7367 - val_loss: 0.4837 - val_accuracy: 0.7750
Epoch 7/20
38/38 [==============================] - 0s 6ms/step - loss: 0.5084 - accuracy: 0.7367 - val_loss: 0.4718 - val_accuracy: 0.7650
Epoch 8/20
38/38 [==

Now lets evaluate our model on our Test dataset

In [27]:
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 3ms/step - loss: 0.5252 - accuracy: 0.7450


[0.5252062678337097, 0.7450000047683716]

Prety Good!